In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [ ]:
%%writefile fairy_interaction_agent.py

from agents.fairy.fairy_state import (
    FairyInteractionState,
    FairyInterationIntentType,
    FairyInterationIntentOutput,
    FairyItemUseOutput,
)
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
from langchain.chat_models import init_chat_model
from enums.LLM import LLM
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from typing import List
from core.common import get_inventory_items
from agents.fairy.util import (
    get_groq_llm_lc
)

llm = get_groq_llm_lc(max_token=40)


def _clarify_intent(query):
    interation_intent_prompt = PromptManager(FairyPromptType.FAIRY_INTERACTION_INTENT).get_prompt(question=query)
    parser_llm = llm.with_structured_output(FairyInterationIntentOutput)
    intent_output: FairyInterationIntentOutput = parser_llm.invoke(interation_intent_prompt)
    return intent_output


def analyze_intent(state: FairyInteractionState):
    last = state["messages"][-1]
    last_message = last.content
    intent_output: FairyInterationIntentOutput = _clarify_intent(last_message)
    return {"intent_types": intent_output.intents}

# LLM Call을 한번이라도 줄이기 위해 의도 분석과 함께 병렬 호출 Node
def create_temp_use_item_id(state: FairyInteractionState):
    last = state["messages"][-1]
    last_message = last.content
    inventory = state["inventory"]
    my_items = get_inventory_items(inventory)
    item_use_prompt = PromptManager(FairyPromptType.FAIRY_ITEM_USE).get_prompt(
        inventory_items=my_items, question=last_message
    )
    parser_llm = llm.with_structured_output(FairyItemUseOutput)
    output: FairyItemUseOutput = parser_llm.invoke(item_use_prompt)
    return {"temp_use_item_id": output.item_id}

def create_interation(state: FairyInteractionState):
    intent_types: List[FairyInterationIntentType] = state["intent_types"]

    item_id = None
    if FairyInterationIntentType.INVENTORY_ITEM_USE in intent_types:
        item_id = state["temp_use_item_id"]

    room_light = 0
    if FairyInterationIntentType.LIGHT_ON_ROOM in intent_types:
        room_light = 1

    if FairyInterationIntentType.LIGHT_OFF_ROOM in intent_types:
        room_light = 2

    isCheckNextRoom = FairyInterationIntentType.MOVE_NEXT_ROOM in intent_types
    
    return {
        "useItemId": item_id,
        "roomLight": room_light,
        "isCheckNextRoom": isCheckNextRoom,
    }


graph_builder = StateGraph(FairyInteractionState)
graph_builder.add_node("analyze_intent", analyze_intent)
graph_builder.add_node("create_temp_use_item_id", create_temp_use_item_id)
graph_builder.add_node("create_interation", create_interation)

graph_builder.add_edge(START, "analyze_intent")
graph_builder.add_edge(START, "create_temp_use_item_id")

graph_builder.add_edge("analyze_intent", "create_interation")
graph_builder.add_edge("create_temp_use_item_id", "create_interation")

graph_builder.add_edge("create_interation", END)
graph = graph_builder.compile()

Overwriting fairy_interaction_agent.py


In [3]:
from langchain_core.messages import HumanMessage
from agents.fairy.fairy_interaction_agent import graph
result = graph.invoke(
    {
        "inventory": [0, 21, 42],
        "messages": [HumanMessage("한손검 사용해주고 방에 불좀 켜줘")],
    }
)


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [5]:
result

{'messages': [HumanMessage(content='한손검 사용해주고 방에 불좀 켜줘', additional_kwargs={}, response_metadata={}, id='8096aeb6-3b0b-451a-8e43-c377b0012a26')],
 'inventory': [0, 21, 42],
 'roomLight': True,
 'isCheckNextRoom': False,
 'useItemId': 0,
 'intent_types': [<FairyInterationIntentType.INVENTORY_ITEM_USE: 'INVENTORY_ITEM_USE'>,
  <FairyInterationIntentType.LIGHT_ON_ROOM: 'LIGHT_ON_ROOM'>],
 'temp_use_item_id': 0}